In [1]:
from matplotlib import pyplot as plt
import pickle
import numpy as np
from scipy.optimize import root, fsolve

import ipywidgets as widgets

from scipy.optimize import root, fsolve

SIMULATIONS_FILE_PREFIX = "simulations_long"

def latex_scientific_notation(value):
    exponent = int(np.log10(value))
    if exponent:
        value = value / 10**exponent
        return f"{value}" + r" \times 10^{" + str(exponent) +"}"
    return f"{value}"

def simulation_label(simulation):
    sim_parms = simulation["parms"]
    result = f"{sim_parms['m0']}, {sim_parms['chosen_depth']}, {latex_scientific_notation(sim_parms['eps'])}"
    result = r"$(m_0, \ell, tol) \rightarrow "+ f"({result})$"
    return result


def obs_labels(obs_name):
    return obs_name

def lieb_robinson_speed(parameters):
    
    f_factor=np.real(max(np.roots(np.poly1d([1, 0, 
                                             -(parameters['Jx']*parameters['Jy']+
                                               parameters['Jx']*parameters['Jy']+
                                               parameters['Jy']*parameters['Jz']), 
                                             -2*parameters['Jx']*parameters['Jy']*parameters['Jz']])
                                 )
                        )
                    )
    chi_y=fsolve(lambda x,y: x*np.arcsinh(x)-np.sqrt(x**2+1)-y, 1e-1, args=(0))[0]
    return 4*f_factor*chi_y

In [2]:
with open(f"{SIMULATIONS_FILE_PREFIX}.pkl", "br") as in_file:
    simulations = pickle.load(in_file)

params = simulations["_params"]
vLR = lieb_robinson_speed(params)


for key, simulation in simulations.items():
    if key=="exact" or key[0]=="_":
        continue
    simulation["velocity_mu"] = np.array(simulation["spectral_norm_Hij_tensor"])
    times_act  = np.array(simulation["saved_cut_times"])
    simulation["times_act"] = times_act
    simulation["velocity_update"] = np.array([1/(times_act[i+1]-times_act[i]) 
                                                          for i in range(len(times_act)-1)])



In [3]:
for name, sim in simulations.items():
    if name =="exact" or name[0]=="_":
        continue
    print(name, sim["date"])

(2,3,0.1,weights) 2024-12-25 15:21:47.183540
(2,5,0.1,weights) 2024-12-25 15:22:18.598295
(2,7,0.1,weights) 2024-12-25 15:53:52.167102
(2,5,0.1,partial_sum) 2024-12-26 12:33:48.352068
(2,7,0.1,partial_sum) 2024-12-26 15:20:02.492115
(3,5,0.1,partial_sum) 2024-12-26 18:10:54.057862
(3,7,0.1,partial_sum) 2024-12-26 19:49:55.689503
(4,5,0.1,partial_sum) 2024-12-27 00:09:24.867974
(4,7,0.1,partial_sum) 2024-12-27 03:48:18.570063
(2,3,0.1,partial_sum) 2024-12-27 15:28:58.017745
(3,3,0.1,partial_sum) 2024-12-27 17:21:16.671258
(4,3,0.1,partial_sum) 2024-12-27 19:22:02.566720
(2,3,0.01,partial_sum) 2024-12-28 10:59:45.927663
(2,7,0.01,partial_sum) 2024-12-28 13:39:20.064955
(4,3,0.01,partial_sum) 2024-12-28 19:47:38.990880
(4,7,0.01,partial_sum) 2024-12-29 00:09:54.246307


In [11]:
import ipywidgets
out = ipywidgets.Output()


def mostrar_observables(sims, obs_lst):
    n = 10
    if len(sims)==0 or len(obs_lst)==0:
        return
    simulation = simulations["exact"]
    t_vlr = np.array(simulation["ev_obs_ex"]["time"]) * vLR
    with out:
        out.clear_output()
        for obs_name in obs_lst:
            if obs_name=="time":
                continue
            obs_label = obs_labels(obs_name)
            simulation = simulations["exact"]
            data = np.array(simulation["ev_obs_ex"][obs_name])
            label = obs_name + " - exact"
            plt.plot(t_vlr, data, label=label, linewidth=2)

            for sim in sims:
                simulation = simulations[sim]
                sim_label = simulation_label(simulation)
                label =  obs_name + " - " + sim_label
                data = np.array(simulation["ev_obs_maxent"][obs_name])
                t_vlr = np.array(simulation["ev_obs_maxent"]["time"]) * vLR
                plt.scatter(t_vlr[::n], data[::n], label=label, linewidth=2)
        plt.legend()
        plt.show()
            
layout=ipywidgets.Layout(width='50%', height='80px', display="flex", align_items = 'stretch')
pick_sim = ipywidgets.SelectMultiple(options=sorted([key for key in simulations.keys() if key!="exact" and key[0]!="_"]))
pick_obs = ipywidgets.SelectMultiple(options=sorted([key for key in simulations["exact"]["ev_obs_ex"].keys() if key!="time"]))
                 
interact=ipywidgets.interactive(mostrar_observables, sims=pick_sim   , obs_lst=pick_obs)

display(
widgets.HBox(children=(pick_obs,pick_sim, out),
                 layout=widgets.Layout()
            #width='50%',
            #grid_template_rows='auto',
            #grid_template_columns='25% 25%',
            #grid_template_areas='''
            #"interact pick_sim  pick_obs"
            #''')
    )
)
# mostrar_observables([1,2],["obs_SzA"])    

In [5]:
import ipywidgets

out = ipywidgets.Output()


def mostrar_local_bound_error(sims):
    n = 10
    if len(sims)==0:
        return
    simulation = simulations["exact"]
    t_vlr = np.array(simulation["ev_obs_ex"]["time"]) * vLR
    for sim in sims:
        simulation = simulations[sim]
        sim_label = simulation_label(simulation)
        label =  "error" + " - " + sim_label
        data = np.array(simulation["local_bound_error"])
        t_vlr = np.array(simulation["ev_obs_maxent"]["time"]) * vLR
        plt.scatter(t_vlr[::n], data[::n], label=label, linewidth=2)
    plt.legend()
            


pick_sim = widgets.SelectMultiple(options=sorted([key for key in simulations.keys() if key!="exact" and key[0]!="_"]))

                 
interact=widgets.interact(mostrar_local_bound_error, sims=pick_sim)
# mostrar_observables([1,2],["obs_SzA"])    

interactive(children=(SelectMultiple(description='sims', options=('(2,3,0.01,partial_sum)', '(2,3,0.1,partial_…

In [6]:
def mostrar_velocidades(sims):
    if len(sims)==0:
        return

    fig, ax = plt.subplots(figsize=(6, 4.5))
    m = 1
    n = 1
    simulation = simulations["exact"]
    timespan = simulation["ev_obs_ex"]["time"]
    t_vlr = np.array(simulation["ev_obs_ex"]["time"]) * vLR            
    for sim in sims:
        simulation = simulations[sim]        
        sim_label = sim #simulation_label(simulation)
        t_vLR = simulation["times_act"][:-1][::m]
        v_ps = simulation["velocity_update"][::m]
        ax.scatter(t_vLR, v_ps, label=sim_label + r" $v_{update}$")
        t_vLR = simulation["times_act"][:-1][::n]
        v_mu = simulation["velocity_mu"][::n]
        ax.plot(t_vLR, v_mu[:len(t_vLR)], label=sim_label + r" $1/\mu$")
    ax.legend()
    # Bordures uniquement à gauche et en bas (style minimaliste)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Personnalisation des ticks
    ax.tick_params(direction='out', length=6, width=2, which='major', grid_alpha=0.6)
    ax.grid(True, which='major', linestyle='--', linewidth=0.75, alpha=0.7)

    ax.legend(frameon=False, loc='center right')
    ax.set_yscale("log")

    # Create a secondary x-axis (twinned) and set its limits
    ax1 = ax.twiny()
    ax1.set_xlim(timespan[0]*vLR, timespan[-1]*vLR)

    # Set axis labels
    ax.set_xlabel(r'$t v_{\rm LR} \, [L]$')
    ax1.set_xlabel(r'$t \, [{\rm arb. units}]$')

    # Format the title using the `params` dictionary values
    # Ensure proper LaTeX formatting and consistent escape sequences
    plt.title(r'$L={}$, $J_x={}$, $v_{{\rm LR}}={}$'.format(
        simulations["_params"]['size'], 
        round(simulations["_params"]['Jx'], 3), 
        round(vLR, 3)
    ))

    # Adjustment final et affichage
    plt.tight_layout()
    plt.show()
            


pick_sim = widgets.SelectMultiple(options=sorted([key for key in simulations.keys() if key!="exact" and key[0]!="_"]))

                 
interact=widgets.interact(mostrar_velocidades, sims=pick_sim)
# mostrar_observables([1,2],["obs_SzA"])    

interactive(children=(SelectMultiple(description='sims', options=('(2,3,0.01,partial_sum)', '(2,3,0.1,partial_…

In [7]:
def mostrar_errores(sims):
    if len(sims)==0:
        return

    fig, ax = plt.subplots(figsize=(8, 4))
    m = 1
    n = 1
    simulation = simulations["exact"]
    timespan = simulation["ev_obs_ex"]["time"]
    t_vlr = np.array(simulation["ev_obs_ex"]["time"]) * vLR            
    for sim in sims:
        simulation = simulations[sim]
        error_func = sim.split(",")[-1][:-1]
        sim_label = error_func + " - " + simulation_label(simulation)
        t_vLR = simulation["times_act"]
        werror = [sum(np.real(errors)**2)**.5   for errors in simulation["instantaneous_w_errors"]]
        ax.plot(t_vLR, werror, label=r"$error bound$ - " + sim_label )
        t_vLR = np.array(simulation["ev_obs_maxent"]["time"]) * vLR            
        lbe = [lbe   for lbe in simulation["local_bound_error"]]
        ax.plot(t_vLR[:-1], lbe, label=r"$lbe$ - " + sim_label )
    ax.legend()
    # Bordures uniquement à gauche et en bas (style minimaliste)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Personnalisation des ticks
    ax.tick_params(direction='out', length=6, width=2, which='major', grid_alpha=0.6)
    ax.grid(True, which='major', linestyle='--', linewidth=0.75, alpha=0.7)

    # ax.legend(frameon=False, loc='center right')

    # Create a secondary x-axis (twinned) and set its limits
    ax1 = ax.twiny()
    ax1.set_xlim(timespan[0]*vLR, timespan[-1]*vLR)

    # Set axis labels
    ax.set_xlabel(r'$t v_{\rm LR} \, [L]$')
    # ax1.set_xlabel(r'$t \, [{\rm arb. units}]$')
    ax.set_ylabel("W")

    # Format the title using the `params` dictionary values
    # Ensure proper LaTeX formatting and consistent escape sequences
    plt.title(r'$L={}$, $J_x={}$, $v_{{\rm LR}}={}$'.format(
        simulations["_params"]['size'], 
        round(simulations["_params"]['Jx'], 3), 
        round(vLR, 3)
    ))

    # Adjustment final et affichage
    plt.tight_layout()
    plt.show()
            


pick_sim = widgets.SelectMultiple(options=sorted([key for key in simulations.keys() if key!="exact" and key[0]!="_"]))

                 
interact=widgets.interact(mostrar_errores, sims=pick_sim)
# mostrar_observables([1,2],["obs_SzA"])    

interactive(children=(SelectMultiple(description='sims', options=('(2,3,0.01,partial_sum)', '(2,3,0.1,partial_…

In [8]:
out_weight = ipywidgets.Output()

def mostrar_peso_errores(sims, idx):
    if len(sims)==0:
        return

    with out_weight:
        out_weight.clear_output()
        fig, ax = plt.subplots(figsize=(8, 4))
        m = 11
        n = 10
        simulation = simulations["exact"]
        timespan = simulation["ev_obs_ex"]["time"]
        t_vlr = np.array(simulation["ev_obs_ex"]["time"]) * vLR            
        for sim in sims:
            simulation = simulations[sim]
            error_func = sim.split(",")[-1][:-1]
            sim_label = error_func + " - " + simulation_label(simulation)
            t_vLR = simulation["times_act"]
            w_a = np.array(simulation["instantaneous_w_errors"]).transpose()
            for i, w_a in enumerate(w_a):
                if i in idx:
                    ax.plot(t_vLR[:len(w_a)], w_a, label=r"$\mathcal{W}_{"+str(i)+"}$" + " - " + sim_label )
        ax.legend()
        # Bordures uniquement à gauche et en bas (style minimaliste)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # Personnalisation des ticks
        ax.tick_params(direction='out', length=6, width=2, which='major', grid_alpha=0.6)
        ax.grid(True, which='major', linestyle='--', linewidth=0.75, alpha=0.7)

        # ax.legend(frameon=False, loc='center right')

        # Create a secondary x-axis (twinned) and set its limits
        ax1 = ax.twiny()
        ax1.set_xlim(timespan[0]*vLR, timespan[-1]*vLR)
        # ax.set_yscale("log")

        # Set axis labels
        ax.set_xlabel(r'$t v_{\rm LR} \, [L]$')
        # ax1.set_xlabel(r'$t \, [{\rm arb. units}]$')
        ax.set_ylabel("W")

        # Format the title using the `params` dictionary values
        # Ensure proper LaTeX formatting and consistent escape sequences
        plt.title(r'$L={}$, $J_x={}$, $v_{{\rm LR}}={}$'.format(
            simulations["_params"]['size'], 
            round(simulations["_params"]['Jx'], 3), 
            round(vLR, 3)
        ))

        # Adjustment final et affichage
        plt.tight_layout()
        plt.show()
            


pick_sim = widgets.SelectMultiple(options=sorted([key for key in simulations.keys() if key!="exact" and key[0]!="_"]))
pick_idx = widgets.SelectMultiple(options=list(range(12)))
interact=widgets.interactive(mostrar_peso_errores, sims=pick_sim, idx=pick_idx)


display(
widgets.HBox(children=(pick_idx, pick_sim, out_weight),
                 layout=widgets.Layout()))

                 
# mostrar_observables([1,2],["obs_SzA"])    

In [9]:
def mostrar_peso_errores_2(sims):
    if len(sims)==0:
        return

    fig, ax = plt.subplots(figsize=(8, 4))
    m = 11
    n = 10
    simulation = simulations["exact"]
    timespan = simulation["ev_obs_ex"]["time"]
    t_vlr = np.array(simulation["ev_obs_ex"]["time"]) * vLR    
    for sim in sims:
        simulation = simulations[sim]
        print(len(simulation["instantaneous_w_errors"]))
        error_func = sim.split(",")[-1][:-1]
        sim_label = error_func + " - " + simulation_label(simulation)
        t_vLR = simulation["times_act"]
        
        w_a = np.array(simulation["instantaneous_w_errors"])[::50]
        for i, w_a in enumerate(w_a):
            ax.plot(w_a, label=r"$\mathcal{W}("+str(t_vLR[50*i])+")$" + " - " + sim_label )
    ax.legend()
    # Bordures uniquement à gauche et en bas (style minimaliste)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Personnalisation des ticks
    ax.tick_params(direction='out', length=6, width=2, which='major', grid_alpha=0.6)
    ax.grid(True, which='major', linestyle='--', linewidth=0.75, alpha=0.7)

    # ax.legend(frameon=False, loc='center right')

    # Create a secondary x-axis (twinned) and set its limits
    ax1 = ax.twiny()
    ax1.set_xlim(timespan[0]*vLR, timespan[-1]*vLR)
    # ax.set_yscale("log")

    # Set axis labels
    ax.set_xlabel(r'$t v_{\rm LR} \, [L]$')
    # ax1.set_xlabel(r'$t \, [{\rm arb. units}]$')
    ax.set_ylabel("W")

    # Format the title using the `params` dictionary values
    # Ensure proper LaTeX formatting and consistent escape sequences
    plt.title(r'$L={}$, $J_x={}$, $v_{{\rm LR}}={}$'.format(
        simulations["_params"]['size'], 
        round(simulations["_params"]['Jx'], 3), 
        round(vLR, 3)
    ))

    # Adjustment final et affichage
    plt.tight_layout()
    plt.show()
            


pick_sim = widgets.SelectMultiple(options=sorted([key for key in simulations.keys() if key!="exact" and key[0]!="_"]))

                 
interact=widgets.interact(mostrar_peso_errores_2, sims=pick_sim)
# mostrar_observables([1,2],["obs_SzA"])    

interactive(children=(SelectMultiple(description='sims', options=('(2,3,0.01,partial_sum)', '(2,3,0.1,partial_…

In [10]:
import json
from datetime import datetime

 
class MyJSONEncoder(json.JSONEncoder):         
    def default(self, o):
        if hasattr(o, "tolist"):
            return o.tolist() # works with any object that has .tolist() method
        if isinstance(o, datetime):
            print("datetime")
            return str(o)
        if isinstance(o, complex):
            return np.real(o)
        # Let the base class default method raise the TypeError
        print(type(o))
        return json.JSONEncoder.default(self, o)
    

#with open("simulations.json", "w") as file_out:
#    json.dump(simulations['(3,3,0.01,weights)'], file_out,
#                           separators=(',', ':'), sort_keys=True, indent=4, cls=MyJSONEncoder)
